In [2]:
import os
import pandas as pd
import numpy as np

1-1. Standardize MT5 Data

In [ ]:
def fix_data(folder_path, fixed_path):
    # Define the standard column names after dropping <VOL> and renaming <TICKVOL>
    standard_columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Spread']
    
    # Create the fixed folder if it doesn't exist
    if not os.path.exists(fixed_path):
        os.makedirs(fixed_path)
    
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            fixed_file_path = os.path.join(fixed_path, filename)
            
            # Read the CSV file
            df = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated values
            
            # Drop the <VOL> column
            df.drop(columns=['<VOL>'], inplace=True)
            
            # Rename <TICKVOL> to 'Vol'
            df.rename(columns={'<TICKVOL>': 'Vol'}, inplace=True)
            
            # Check if the <TIME> column exists
            if '<TIME>' in df.columns:
                # Combine <DATE> and <TIME> into a single 'Datetime' column
                df['Datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'], format='%Y.%m.%d %H:%M:%S')
                # Drop the original <DATE> and <TIME> columns
                df.drop(columns=['<DATE>', '<TIME>'], inplace=True)
            else:
                # If no <TIME> column, just convert <DATE> to datetime
                df['Datetime'] = pd.to_datetime(df['<DATE>'], format='%Y.%m.%d')
                # Drop the original <DATE> column
                df.drop(columns=['<DATE>'], inplace=True)
            
            # Reorder columns to ensure 'Datetime' is the first column
            df = df[['Datetime'] + [col for col in df.columns if col != 'Datetime']]
            df['Datetime'].to_date
            
            # Rename other columns to standard names
            df.columns = standard_columns
            
            # Save the corrected DataFrame to the fixed folder
            df.to_csv(fixed_file_path, index=False)
            
            print(f"Processed and saved: {filename}")

# Example usage
folder_path = '../data/mt5'
fixed_path = '../data/gold'
fix_data(folder_path, fixed_path)

Processed and saved: Daily_201708100000_202503140000.csv
Processed and saved: H1_201708100000_202503142200.csv
Processed and saved: H4_201708100000_202503142000.csv
Processed and saved: M15_202012161445_202503131615.csv
Processed and saved: M2_202408190410_202503142258.csv
Processed and saved: M5_202310121825_202503142255.csv


1-2. Add Features

In [5]:
def calculate_atr(df, period=14):
    """Calculate the Average True Range (ATR) for a given DataFrame."""
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift())
    low_close = abs(df['Low'] - df['Close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr

def add_features(df):
    """Add features like day of the week, ATR, and EMAs to the DataFrame."""

    # Calculate Price Change Value
    df['PriceChange'] = df['Close'].diff()
    # Calculate Price Change Percentage
    df['PriceChangePct'] = df['Close'].pct_change() * 100

    # Add day of the week (0 = Monday, 6 = Sunday)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df['DayOfWeek'] = df['Datetime'].dt.dayofweek
    
    # Calculate ATR (Average True Range)
    df['ATR'] = calculate_atr(df)
    
    # Calculate EMAs (Exponential Moving Averages)
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_100'] = df['Close'].ewm(span=100, adjust=False).mean()
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()

    # Calculate rolling volatility
    df['volatility'] = df['PriceChange'].rolling(window=20).std()

    # Define regimes
    df['regime'] = np.where(df['volatility'] > 3, 'high_vol', 'low_vol')
        
    return df

df = pd.read_csv('..\\data\\gold\\M15_202012161445_202503131615.csv')
features = add_features(df)

# Drop first Rows so we have valid features
features.drop(df.index[:221], inplace=True)

# Save full data
features.set_index('Datetime', inplace=True)
features.to_csv('..\\data\\M15_full.csv')